<a href="https://colab.research.google.com/github/darknightush/Resume_Parser/blob/Master/Resume_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resume Parser

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import pickle
import random

In [5]:
data = pickle.load(open('data.pkl','rb'))
data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [6]:
# Load Blank Model
nlp = spacy.blank('en')

def training(data):
    # Remove all pipelines and add NER pipeline from the model
    if 'ner'not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        # adding NER pipeline to nlp model
        nlp.add_pipe(ner,last=True)
    
    #Add labels in the NLP pipeline
    for _, annotation in data:
        for ent in annotation.get('entities'):
            ner.add_label(ent[2])
    
    #Remove other pipelines if they are there
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(30): # train for 10 iterations
            print("Starting iteration " + str(itn))
            random.shuffle(data)
            losses = {}
            index = 0
            for text, annotations in data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass
                
            print(losses)
# Start Training model
training(data)

Starting iteration 0
{'ner': 15231.471876251697}
Starting iteration 1
{'ner': 10172.174889678263}
Starting iteration 2
{'ner': 8747.356816184034}
Starting iteration 3
{'ner': 6749.569263962965}
Starting iteration 4
{'ner': 6244.544416655366}
Starting iteration 5
{'ner': 5397.981417064491}
Starting iteration 6
{'ner': 5446.890278133104}
Starting iteration 7
{'ner': 5473.634519398004}
Starting iteration 8
{'ner': 5081.475572136505}
Starting iteration 9
{'ner': 4303.01603472484}
Starting iteration 10
{'ner': 4672.850855090795}
Starting iteration 11
{'ner': 3386.8970842396193}
Starting iteration 12
{'ner': 4433.490901241592}
Starting iteration 13
{'ner': 5484.845435524629}
Starting iteration 14
{'ner': 4369.370967092233}
Starting iteration 15
{'ner': 5131.341105034062}
Starting iteration 16
{'ner': 3581.448207004029}
Starting iteration 17
{'ner': 3352.329272256251}
Starting iteration 18
{'ner': 2786.8149138532644}
Starting iteration 19
{'ner': 3215.345655666321}
Starting iteration 20
{'ner

In [7]:
nlp.to_disk('nlp_ner_model')

In [8]:
#Loading Model
nlp_model = spacy.load('nlp_ner_model')

In [10]:
# trying and seeing the prediction of the model
doc = nlp_model(data[0][0])
for ent in doc.ents:
    print(f"{ent.label_.upper():{30}}-{ent.text}")

NAME                          -Arpit Jain
DESIGNATION                   -Quality Analyst
COMPANIES WORKED AT           -ThoughtWorks Technologies
EMAIL ADDRESS                 -indeed.com/r/Arpit-Jain/3714fe32f98b03a9
DESIGNATION                   -Quality Analyst
DESIGNATION                   -Quality Analyst
COMPANIES WORKED AT           -ThoughtWorks Technologies
COMPANIES WORKED AT           -ThoughtWorks Technologies
DESIGNATION                   -Quality Analyst
COMPANIES WORKED AT           -ThoughtWorks Technologies
DEGREE                        -B.Tech
COLLEGE NAME                  -Jaypee Institute Of Information Technology
SKILLS                        -SKILLS  Manual Testing, Protractor, Selenium Webdriver, Automation Testing, API


In [11]:
!pip install PyMuPDF 


     |████████████████████████████████| 8.7 MB 4.8 MB/s 


In [18]:
import sys, fitz
fname = 'Smith Resume.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text + str(page.get_text())
art = " ".join(text.split('\n'))
print(art)

Michael Smith  BI / Big Data/ Azure  Manchester, UK- Email me on Indeed: indeed.com/r/falicent/140749dace5dc26f    10+ years of Experience in Designing, Development, Administration, Analysis,  Management  inthe  Business  Intelligence  Data  warehousing,  Client  Server  Technologies, Web-based Applications, cloud solutions and Databases.  Data warehouse: Data analysis, star/ snow flake schema data modeling and design  specific todata warehousing and business intelligence environment.  Database: Experience in database designing, scalability, back-up and recovery,  writing andoptimizing SQL code and Stored Procedures, creating functions, views,  triggers and indexes.   Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL  Azure, StreamAnalytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure  data lake analytics(U-SQL).  Big Data: Worked Azure data lake store/analytics for big data processing and Azure  data factoryto schedule U-SQL jobs. Designed and d

In [19]:
# Applying the model
doc = nlp_model(art)
list1=[]
list2 = []
for ent in doc.ents:
    list1.append(ent.label_.upper())
    list2.append(ent.text)
dictionary = dict(zip(list1, list2))
dictionary

{'COMPANIES WORKED AT': 'Microsoft',
 'EMAIL ADDRESS': 'indeed.com/r/falicent/140749dace5dc26f',
 'NAME': 'Michael Smith',
 'SKILLS': 'problem solving (Less than 1 year), project lifecycle (Less than 1 year), project'}

In [20]:
import json 
      
# Serializing jsona  
json_object = json.dumps(dictionary, indent = 4) 
print(json_object)

{
    "NAME": "Michael Smith",
    "EMAIL ADDRESS": "indeed.com/r/falicent/140749dace5dc26f",
    "COMPANIES WORKED AT": "Microsoft",
    "SKILLS": "problem solving (Less than 1 year), project lifecycle (Less than 1 year), project"
}
